In [1]:
import re
import string
import pandas as pd
import numpy as np

import gensim
# from gensim.models import Word2Vec
# from gensim.scripts.glove2word2vec import glove2word2vec
# import gensim.downloader as api
from gensim.parsing.preprocessing import remove_stopwords

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nccru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nccru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
df = pd.read_csv('data/song_lyrics_clean.csv', nrows=1000)

In [ ]:
# df.head()

In [ ]:
### clean up current lyrics
### remove chorus:, intro:, etc.
### stop words, punctuation

In [4]:
def remove_between_brackets(text):
  """Removes all text between any matching pair of brackets, including the brackets themselves."""
  return re.sub(r'\[.*?\]', '', text)

In [5]:
def remove_punctuation(text):
    """Removes punctuation from a string."""
    return text.translate(str.maketrans('', '', string.punctuation))

In [6]:
def cleanse_lyrics(df):

    df['cleaned_lyrics'] = df['lyrics'].apply(remove_between_brackets)
    df['cleaned_lyrics'] = df['cleaned_lyrics'].apply(remove_punctuation)
    df['cleaned_lyrics'] = df['cleaned_lyrics'].str.lower()
    df['cleaned_lyrics'] = df['cleaned_lyrics'].str.replace(",", '')
    df['cleaned_lyrics'] = df['cleaned_lyrics'].apply(remove_stopwords)
    df['tokenized_text'] = df["cleaned_lyrics"].apply(word_tokenize)

    return df

In [7]:
df = cleanse_lyrics(df)
df.head()

,lyrics,genre,cleaned_lyrics,tokenized_text
0,she got a broke down el camino in the front ya...,country,got broke el camino yard blocks mom walks pink...,"[got, broke, el, camino, yard, blocks, mom, wa..."
1,later on when weve all come down and the stree...,country,later weve come streets funeral event falling ...,"[later, weve, come, streets, funeral, event, f..."
2,longing for something more will i ever be free...,country,longing free free waitin sunshine door free oh...,"[longing, free, free, waitin, sunshine, door, ..."
3,my heart beats with the lonely rain wishin i c...,country,heart beats lonely rain wishin face change dia...,"[heart, beats, lonely, rain, wishin, face, cha..."
4,an old cowpoke went ridin out one dark and win...,country,old cowpoke went ridin dark windy day rested s...,"[old, cowpoke, went, ridin, dark, windy, day, ..."


In [ ]:
# # clean your sentences
# stopwords = [YOUR_STOPWORDS_HERE]
# cleaned_sentences = []
# for sentence in sentences:
#   cleaned = [word.lower() for word in sentence]
#   cleaned = [word for word in cleaned if word not in stopwords]
#   cleaned_sentences.append(cleaned)

# build a word2vec model on your dataset
# sentences = df['tokenized_text'].tolist()
# base_model = Word2Vec(vector_size=100, min_count=5)
# base_model.build_vocab(sentences)
# total_examples = base_model.corpus_count

In [ ]:
# base_model.train(sentences, total_examples=total_examples, epochs=base_model.epochs) 

In [ ]:
# list(w for w in base_model.wv.index_to_key)[:5]

In [ ]:
# base_model.wv.vectors[:5]

In [ ]:
# def apply_word2vec(sentences):
#   """
#   apply_word2vec
#   params: sentences -> 'tokenized_text'
#   returns: word2vec model
  
#   Access vectors from base_model.wv.vectors and base_model.wv.index_to_key
#   """
#   base_model = Word2Vec(vector_size=100, min_count=5)
#   base_model.build_vocab(sentences)
#   # base_model.train(sentences, total_examples=base_model.corpus_count, epochs=base_model.epochs) 
#   return base_model

In [ ]:
# model = apply_word2vec(df)

### Embedding: GloVe

In [ ]:
import numpy as np
import torch
def create_glove_matrix(df):
    '''
    Returns: df -> pandas dataframe with column of vectors for each word in the song
    '''

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df['cleaned_lyrics'])

    max_length = max(len(data) for data in df['cleaned_lyrics'])
    word_index = tokenizer.word_index
    vocab_size = len(word_index)    

    # padding text data
    sequences = tokenizer.texts_to_sequences(df['cleaned_lyrics'])
    padded_seq = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

    # create embedding index
    embedding_index = {}
    with open('glove.42B.300d.txt', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = coefs

    # # create embedding matrix
    # embedding_matrix = np.zeros((vocab_size+1, 300))
    # for word, i in word_index.items():
    #     embedding_vector = embedding_index.get(word)
    #     if embedding_vector is not None:
    #         embedding_matrix[i] = embedding_vector

    word_vec_tensor = []
    for index, row in df.iterrows():
        word_vecs = []
        for word in df['tokenized_text'][index]:
            word_vec = embedding_index.get(word)
            word_vecs.append(word_vec)

        word_vec_tensor.append(word_vecs)

    return word_vec_tensor

In [9]:
create_glove_matrix(df)

,lyrics,genre,cleaned_lyrics,tokenized_text
0,she got a broke down el camino in the front ya...,country,got broke el camino yard blocks mom walks pink...,"[got, broke, el, camino, yard, blocks, mom, wa..."
1,later on when weve all come down and the stree...,country,later weve come streets funeral event falling ...,"[later, weve, come, streets, funeral, event, f..."
2,longing for something more will i ever be free...,country,longing free free waitin sunshine door free oh...,"[longing, free, free, waitin, sunshine, door, ..."
3,my heart beats with the lonely rain wishin i c...,country,heart beats lonely rain wishin face change dia...,"[heart, beats, lonely, rain, wishin, face, cha..."
4,an old cowpoke went ridin out one dark and win...,country,old cowpoke went ridin dark windy day rested s...,"[old, cowpoke, went, ridin, dark, windy, day, ..."
...,...,...,...,...
995,no one will ever know my heart is breaking alt...,country,know heart breaking million teardrops start fl...,"[know, heart, breaking, million, teardrops, st..."
996,i dont just love you i live you where im conce...,country,dont love live im concerned comes lifetimes id...,"[dont, love, live, im, concerned, comes, lifet..."
997,i got kicked off noahs ark i turn my cheek to ...,country,got kicked noahs ark turn cheek unkind remarks...,"[got, kicked, noahs, ark, turn, cheek, unkind,..."
998,if you miss the train im on you will know that...,country,miss train im know gone hear whistle blow mile...,"[miss, train, im, know, gone, hear, whistle, b..."


In [ ]:
assert 1==2

In [ ]:
# word_vecs = []
# for word in df['tokenized_text'][0]:
#     word_vec = embedding_index.get(word)
#     word_vecs.append(word_vec)

### Embedding: BERT and DistilBERT

In [ ]:
#import library
from transformers import DistilBertTokenizer, DistilBertModel
import torch
# import numpy as np

#load DistilBERT tokenizer and a pretrained model to avoid training from scratch
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [ ]:
#tokenize and obtain embeddings
def get_lyrics_embedding(lyrics):
    tokens = tokenizer(
        lyrics,
        truncation = True,
        padding = True, 
        max_length = 512, #DistilBERT has a max token limit of 512
        return_tensors = "pt" #converting output as PyTorch since DistilBERT expects tensors not token IDs
        )
    
    with torch.no_grad(): #removes gradient calculation to save memory usage since inferences are not needed as we are predicting, not training
        output = model(**tokens)

    cls_embedding = output.last_hidden_state[:, 0] #extract first token with CLS
    cls_embedding = cls_embedding.detach() #detach from PyTorch's gradient computation 
    cls_embedding = cls_embedding.cpu() #converting tensor to CPU to ensure compatability (ie. NumPy array conversion)
    cls_embedding = cls_embedding.squeeze() #remove any extra dimensions
    
    embedding = cls_embedding.numpy() #converting into NumPy array
    return embedding

#converting lyrics into embeddings using nrows
embeddings = np.array([get_lyrics_embedding(lyric) for lyric in df['lyrics']])


### Bag of Words Model

### TF-IDF

### Question: Are we using Keras / PyTorch?
This may change the format and implementation of the current method of embedding.